In [1]:
import os
import time
import torch

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from models import ShallowNN
from utils import load_file, get_all_possible_pairs
from evals import evaluate, pairwise_euclidean_distance , influence, calculate_hessian_flattened

features = 197
batch_size = 64
loss_fn = torch.nn.L1Loss()

In [2]:
client_ids = ["0_0","0_1","0_2","0_3","0_4","0_5","1_0","1_1","1_2","1_3","1_4","1_5","2_0","2_1","2_2","2_3","2_4","2_5","3_0","3_1","3_2","3_3","3_4","3_5"]

In [3]:
global_model = ShallowNN(features)
global_model.load_state_dict(torch.load('checkpt/epoch_500/_fedl_global_500.pth'))

<All keys matched successfully>

In [4]:
from pyhessian import hessian

In [5]:
eigen_values = []
eigen_vectors = {}
for client in client_ids:
    val_set = torch.load("trainpt/"+str(client)+".pt")
    data_loader = DataLoader(val_set, 64, shuffle = True)
    hessian_comp = hessian(global_model, loss_fn,  dataloader=data_loader, cuda=False)
    top_eigenvalues, top_eigenvector = hessian_comp.eigenvalues()
    print(top_eigenvalues)
    eigen_values.append(round(top_eigenvalues[-1],4))
    eigen_vectors[client] = top_eigenvector

/Users/tharuka/anaconda3/envs/fedce/lib/python3.11/site-packages/torch/autograd/__init__.py:200: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/engine.cpp:1156.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


[1.387229084968567]
[1.3263691663742065]
[-1.6219123601913452]
[2.911726951599121]
[1.044628381729126]
[1.4119017124176025]
[1.049511432647705]
[1.052514672279358]
[0.5787808895111084]
[1.1117876768112183]
[-0.4282904863357544]
[1.5827229022979736]
[0.8526875972747803]
[-0.8462137579917908]
[1.009723424911499]
[0.6890099048614502]
[-1.2222977876663208]
[-1.3032875061035156]
[0.8303232789039612]
[-1.0736682415008545]
[0.5403720140457153]
[-0.9259750843048096]
[0.3718031942844391]
[-1.434649109840393]


In [6]:
proxi = []
for client in client_ids:
    client_vector = eigen_vectors[client][0][0]
    total_dist = 0.0
    for key,vector in eigen_vectors.items():
        if key != client:
            distance = np.linalg.norm(vector[0][0] - client_vector)
            total_dist += distance
    proxi.append(total_dist)

In [7]:
eigens = {"client_id": client_ids, "eigen_values":eigen_values}
eigens = pd.DataFrame(eigens)
eigens.to_csv("insights/eigens.csv" , index=False)